In [1]:
import numpy as np
np.seterr(all='raise')
import sys
sys.path.append('../')

from meshmaker.base import Base, Laziness
from meshmaker.model import Model
from meshmaker.mesh import Mesh
from meshmaker.pmesh import ParamMesh, MetaMesh, MetaScene
#from meshmaker.meta import Railing, Stairs
from meshmaker.tform import TForm
from meshmaker.vec3 import vec3
from meshmaker.quat import quat
from meshmaker.loop import loop as Loop
from meshmaker.delaunay import triangulation
from meshmaker.planargraph import planargraph
#from meshmaker.geometry import batch, slide, loop_offset, loop_normal, loop_contains, loop_split, isnear, near, loopO
from meshmaker.geometry import batch, slide, isnear, near
from meshmaker.mgl import show, MainShader, EdgeShader, WireShader, LazyMaterials
from meshmaker.plt import *
from collections import defaultdict
from functools import partial, reduce
from itertools import cycle
import json

show = partial(show, programs=[MainShader(), EdgeShader(), WireShader(color=vec3.U(0.1))], background=vec3.U(0.5))

In [ ]:
a = Mesh.cube_mesh()
b = Mesh.cube_mesh()
vec3.U(-0.5).trnps(a.vertices)
vec3.U( 0.5).trnps(b.vertices)
show(TForm(models=[Model(meshes={'generic_11': [a], 'generic_10': [b]})]))
#show([a, b])

Loaded texture: generic_11 (../resources/textures/generics/generic_11.png)
Loaded texture: generic_10 (../resources/textures/generics/generic_10.png)
(0, 260, 958, 1059)
(0, 260, 958, 538)
(958, 1059)
> /home/cogle/dev/mesh_maker/meshmaker/mgl.py(403)key_event()
-> screenshot.create(self.ctx.fbo)


In [4]:
show(a.union(b))
show(b.union(a))
show(a.difference(b))
show(b.difference(a))
show(a.intersect(b))
show(b.intersect(a))

Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)


# use CSG to design control meshes which cover 3d volume

In [10]:
class Partition(Base):
    
    def __init__(self, *meshes, **kws):
        super().__init__(**kws)
        self.meshcount = 0
        self.meshes = []
        for mesh in meshes:
            self.av(mesh)
    
    def __iter__(self):
        """Yield the index, mesh pairs in the partition"""
        for i, mesh in enumerate(self.meshes):
            if mesh is not None:
                yield (i, mesh)
    
    def av(self, mesh, **kws):
        """Add new volume/vertex"""
        m = len(self.meshes)
        self.meshes.append(mesh)
        self.meshcount += 1
        return m
    
    def rv(self, v):
        """Remove volume/vertex v"""
        mesh = self.meshes[v]
        self.meshes[v] = None
        self.meshcount -= 1
        return mesh
    
    def sv(self, O, N, *vs):
        """Split vertices *vs using plane defined by O and N"""
        nvs = []
        for v in vs:
            v = self.rv(v)
            x, y = v.split(O, N)
            nvs.append(self.av(x))
            nvs.append(self.av(y))
        return nvs
    
    def graph(self, Interface):
        """Map the adjacency of volumes within the partition
        identifying the intersections of their bounding faces"""
        adjacent = defaultdict(lambda : defaultdict(lambda : {}))
        normals = {i: mesh.face_normals() for i, mesh in self}
        for i, u in self:
            for k, uF in u:
                uN = normals[i][k]
                uL = [u.vertices[x].cp() for x in uF]
                uL.reverse()
                for j, v in self:
                    if i < j:
                        continue
                    for l, vF in v:
                        vN = normals[j][l]
                        vL = [v.vertices[x].cp() for x in vF]
                        if uN.isnear(vN.fp()):
                            if isnear(uL[0].dot(uN), vL[0].dot(uN)):
                                #overlap = loop_intersection(uL, vL)
                                overlap = [olap.ps for olap in Loop(uL).intersect(Loop(vL))]
                                overlap = Interface(overlap)
                                if overlap:
                                    adjacent[i][j][k] = overlap
                                    adjacent[j][i][l] = overlap
        return adjacent

In [24]:
def Wall(interfaces, texture):

    def setportals(loop, ploops):
        """Make copies of ploops which are properly coplanar with loop"""
        lN = Loop(loop).N
        aligned = []
        for ploop in ploops:
            ploop = [p.cp() for p in ploop]
            pN = Loop(ploop).N
            if not lN.isnear(pN):
                ploop.reverse()
            inset = [p.cp() for p in ploop]
            dN = lN * (loop[0].dot(lN) - ploop[0].dot(lN))
            if not isnear(dN.mag(), 0):
                dN.trnps(ploop)
                aligned.append((ploop, inset))
            else:
                aligned.append((ploop, None))
        return aligned

    def meta(control, faces):
        
        # TODO: support different types of interface...
        #       closed, open, 1+ doors/windows
        
        assert len(faces) == 1
        meshes = defaultdict(list)
        for f in faces:
            loop = [control.vertices[v] for v in control.faces[f]]

            ploops = []
            for interface in interfaces:
                portals = interface['portals']
                for portal, support in setportals(loop, portals):
                    ploops.append(portal)
                    if support is not None:
                        mesh = Mesh()
                        mesh.bridge(portal, support)
                        meshes[interface['texture']].append(mesh)

                for sloop in interface['support']:
                    dbg = Mesh.cube_mesh(0.1)
                    N = Loop(loop).N * 0.1
                    (vec3.com(sloop) + N).trnps(dbg.vertices)
                    meshes['generic_17'].append(dbg)

            mesh = Mesh()

            #mesh.af(loop)
            #for ploop in ploops:
            #    mesh.af(ploop)

            try:
                py = (
                    [p.cp().quant() for p in loop],
                    [[p.cp().quant() for p in ib] for ib in ploops])
                mesh.apy(py, 0.0001, None)
            except:
                print(((loop, ploops), ))

            meshes[texture].append(mesh)

        return TForm(models=[Model(meshes=meshes)])

    return meta


def show_graph(nodes, edges):
    """Represent topology of a graph of mesh nodes"""
    # nodes is a dict of (int, mesh) pairs
    # edges is a list of (i, j) int pairs, indices in nodes
    meshes = defaultdict(list)
    for i, node in nodes.items():
        cube = Mesh.cube_mesh(0.1)
        vec3.com(node.vertices).trnps(cube.vertices)
        meshes['generic_8'].append(cube)
    def wires(tf, nodes=None, edges=None):
        wires = []
        for i, j in edges:
            wires.append((vec3.com(nodes[i].vertices),
                          vec3.com(nodes[j].vertices)))
        return wires        
    meshes[next(meshes.keys().__iter__())][0].wires =\
        partial(wires, nodes=nodes, edges=edges)
    return TForm(models=[Model(meshes=meshes)])


texturecycle = cycle((f'generic_{i}' for i in range(8, 14)))
nexttex = lambda : next(texturecycle)


def show_partition(partition, wallwidth=0.2):
    shell = Mesh.Union(*filter(None, partition.meshes)).fp()
    shell = partition.av(shell) # TODO: avoid modifying input...
    #shell = None
    
    def Interface(support):
        interface = dict(support=support, portals=[])
        interface['portals'].extend(
            [Loop(loop).offset(0.5).ps for loop in support])
        return interface
    
    adjacent = partition.graph(Interface)
    
    ptexture = nexttex()

    nodes = {}
    for i in adjacent:
        texture = nexttex()
        nodes[i] = partition.meshes[i].cp().fp()
        nodes[i].offset(-wallwidth * 2 if i == shell else -wallwidth)
        for k, face in nodes[i]:
            face = [partition.meshes[i].vertices[v].cp() for v in face]
            interfaces = []
            for j in adjacent[i]:
                if k in adjacent[i][j]:
                    #support = adjacent[i][j][k]
                    interface = adjacent[i][j][k]
                    interface['texture'] = ptexture
                    #if not interface['portals']:
                    #    print('addem')
                    #    portals = [Loop(loop).offset(0.5).ps for loop in interface['support']]
                    #    interface['portals'].extend(portals)
                    #else:
                    #    print('recycleem')
                    #interface = {'support': support,
                    #             'texture': texture,
                    #             'portals': [Loop(loop).offset(0.5).ps for loop in support]}
                    interfaces.append(interface)
            print(i, k, len(interfaces))
            wall = Wall(interfaces, texture)
            wall.__name__ = f'metawall-<{i},{k}>'
            nodes[i].meta[k] = wall
    edges = [(i, j) for i in adjacent for j in adjacent[i]]

    topology = show_graph(nodes, edges)

    #shell_tex = 'generic_3'
    #shell = BSP.brep_union(*filter(None, self.meshes))
    #shell = shell.offset(-0.1)
    #meshes[shell_tex].append(shell)
    #shell = Mesh.Union(*filter(None, partition.meshes))
    #shell = shell.offset(-0.1)
    #for k, _ in shell:
    #    shell.meta[k] = 'generic_3'
    #nodes[-1] = shell
    
    mr = TForm(metas=[MetaMesh(node) for i, node in nodes.items()])
    mr.add(topology)
    ms = MetaScene(mr)
    return ms


r = 4
control = Mesh.cube_mesh(r)
p = Partition()
x = p.av(control)
x, y = p.sv(vec3.Z(1), vec3.Z(), x)
x, z = p.sv(vec3.Z(1), vec3.X(), x)
w, z = p.sv(vec3.Z(1), vec3.Y(), z)
y, a = p.sv(vec3.Z(1), vec3.U().xy().nrm(), y)
p.rv(w)
p.rv(y)
ms = show_partition(p)
show(ms)

6 0 1
6 1 1
6 2 1
6 3 2
6 4 1
6 5 1
3 0 1
3 1 1
3 2 1
3 3 1
3 4 2
3 5 2
8 0 1
8 1 1
8 2 1
8 3 3
8 4 1
9 0 3
9 1 3
9 2 3
9 3 1
9 4 1
9 5 2
9 6 2
9 7 3
9 8 2
9 9 2
9 10 1
9 11 1
9 12 1
9 13 2
9 14 1
(([vec3(3.8877, -3.9535, 0.8412), vec3(-3.8845, -3.8845, 0.8845), vec3(-3.9535, 3.8877, 0.8412)], [[vec3(0.4991, -1.2080, 0.8448), vec3(0.4991, -3.5009, 0.8448), vec3(2.7920, -3.5009, 0.8448)], [vec3(-0.5008, -3.5008, 0.8633), vec3(-0.5008, -0.5008, 0.8633), vec3(-3.5008, -0.5008, 0.8633), vec3(-3.5008, -3.5008, 0.8633)], [vec3(-1.2080, 0.4991, 0.8448), vec3(-3.5009, 0.4991, 0.8448), vec3(-3.5009, 2.7920, 0.8448)]]),)
Loaded texture: generic_11 (../resources/textures/generics/generic_11.png)
Loaded texture: generic_12 (../resources/textures/generics/generic_12.png)
Loaded texture: generic_10 (../resources/textures/generics/generic_10.png)
Loaded texture: generic_17 (../resources/textures/generics/generic_17.png)
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)
Loaded t

In [4]:
p.meshes

[None,
 None,
 None,
 None,
 None,
 None,

In [6]:
# break edges where topologically sensical

r = 4
control = Mesh.cube_mesh(r)
p = Partition()
x = p.av(control)
x, y = p.sv(vec3.Z(1), vec3.Z(), x)
x, z = p.sv(vec3.Z(1), vec3.X(), x)
w, z = p.sv(vec3.Z(1), vec3.Y(), z)
y, a = p.sv(vec3.Z(1), vec3.U().xy().nrm(), y)
p.rv(w)
p.rv(y)

shell = Mesh.Union(*filter(None, p.meshes))
print(shell.faces)

def clean(self):
    for f, face in self:
        print(f, len(face))
    return self

shell = clean(shell)

shell.offset(1)

show(TForm(models=[Model(meshes={'generic_8': [shell]})]))

[[0, 1, 2, 3], [4, 0, 3, 5], [4, 6, 7, 1], [1, 8, 9, 2], [8, 10, 11, 9], [3, 2, 9, 11], [12, 5, 3, 13], [10, 8, 1, 14], [15, 16, 14], [15, 17, 6, 4], [17, 7, 6], [1, 7, 17, 15], [13, 11, 10, 14], [16, 4, 5, 12], [14, 16, 12, 13]]
0 4
1 4
2 4
3 4
4 4
5 4
6 4
7 4
8 3
9 4
10 3
11 4
12 4
13 4
14 4
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)


# TODO: support layers/groups for showing/hiding meshes in mgl window 